In [1]:
!pip install scipy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
df = pd.read_csv("/Users/jeremyhudsonchan/Dropbox/Files/Github_Repos/Twitter-Sentiment-Analysis/data/processed/training.1600000.processed.noemoticon.csv", encoding = "latin-1", low_memory=False)

In [4]:
df.head()

,ids,date,user,text,target
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...,0
1,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...,0
2,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,0
3,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all....",0
4,1467811372,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew,0


In [5]:
# randomly sample 25000 rows with target == 0, and 25000 rows with target == 4
neg_df = df[df.target == 0].sample(n=25000, random_state=5)
pos_df = df[df.target == 4].sample(n=25000, random_state=5)

In [6]:
neg_df.head()

,ids,date,user,text,target
740007,2265878782,Sun Jun 21 07:29:15 PDT 2009,MeeJong,This morning my daughter asked me if I hate my...,0
562036,2205565064,Wed Jun 17 05:00:35 PDT 2009,kathysyahrizal,"I forgot to charge my bb, zzzzz only 35% left",0
318661,2002872654,Tue Jun 02 05:22:07 PDT 2009,kgautam,my MBP battery is fluctuating between dead to ...,0
131276,1835496893,Mon May 18 06:07:33 PDT 2009,thejanice,i really wish my landlord would call me back. ...,0
39699,1573596472,Tue Apr 21 01:55:46 PDT 2009,lucyxechelon,mean blog people,0


In [7]:
pos_df.head()

,ids,date,user,text,target
1129018,1975455456,Sat May 30 15:11:36 PDT 2009,MELi_BELLY,is celebrating people that I love! --&gt; Lov...,4
1075997,1967341404,Fri May 29 19:24:16 PDT 2009,Vinamae,with my babe. bye!,4
1153687,1978844458,Sun May 31 00:40:03 PDT 2009,BrinaM,Morning all. Gorgeous sunny day again here and...,4
1388967,2053073343,Sat Jun 06 03:09:58 PDT 2009,CrashLiverar,@JoanneDuran It looks like you made 900 follow...,4
1341127,2031190204,Thu Jun 04 09:27:46 PDT 2009,eyaustin,@LynzMichelle Its exciting because we had a lo...,4


In [8]:
# combine neg_df and pos_df
use_df = pd.concat([neg_df, pos_df], axis=0)

In [9]:
pd.unique(use_df.target)

array([0, 4])

In [10]:
# print number of 0s and 4s
print("Number of 0s: ", len(use_df[use_df.target == 0]))
print("Number of 4s: ", len(use_df[use_df.target == 4]))

Number of 0s:  25000
Number of 4s:  25000


In [11]:
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [12]:
# use pretrained google word2vec model
model_word2vec = gensim.models.KeyedVectors.load_word2vec_format("/Users/jeremyhudsonchan/Dropbox/Files/Github_Repos/Twitter-Sentiment-Analysis/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [13]:
# apply model_word2vec to each text in use_df
def word2vec(text):
    text = text.split()
    # use keyedvector's .get_vector() method to get the vector for each word in text
    # if word is not in model_word2vec, return a vector of 0s
    text = [model_word2vec.get_vector(word) if word in model_word2vec else np.zeros(300) for word in text]
    return text

# apply word2vec to each text in use_df
use_df['text'] = use_df['text'].apply(word2vec)

In [14]:
print(use_df['text'].head(1))

740007    [[-0.2890625, 0.19921875, 0.16015625, 0.025268...
Name: text, dtype: object


In [15]:
# sum all the vectors in each text
def sum_vectors(text):
    text = np.array(text)
    text = np.sum(text, axis=0)
    return text

# apply sum_vectors to each text in use_df
use_df['text'] = use_df['text'].apply(sum_vectors)

In [16]:
print(use_df['text'].head(2))

740007    [0.6248779296875, 0.260833740234375, 1.0648345...
562036    [0.875732421875, 0.374603271484375, 0.68841552...
Name: text, dtype: object


In [17]:
print(len(use_df['text']))

50000


In [18]:
# check if each text has 300 elements
for i in range(len(use_df['text'])):
    if len(use_df['text'].iloc[i]) != 300:
        print(i)

In [25]:
# make use_df['text'] to a pandas dataframe with 300 columns, keeping the indexs
temp_df = pd.DataFrame(use_df['text'].tolist(), index=use_df.index)
# add target column to temp_df
temp_df['target'] = use_df['target']

In [26]:
temp_df['target'].head()

740007    0
562036    0
318661    0
131276    0
39699     0
Name: target, dtype: int64

In [27]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(temp_df.drop('target', axis=1), temp_df['target'], test_size=0.2, random_state=5)

In [29]:
# logistic regression, with train_x being a list of lists
from sklearn.linear_model import LogisticRegression
def logistic_regression(train_x, train_y):
  classifier = LogisticRegression(max_iter=100000)
  classifier.fit(train_x, train_y)
  return classifier

# svm
from sklearn.svm import SVC
def svm(train_x, train_y):
    # take a subset of train_x and train_y
    classifier = SVC(max_iter=100000, C=1.0)
    classifier.fit(train_x, train_y)
    return classifier

# knn
from sklearn.neighbors import KNeighborsClassifier
def knn(train_x, train_y):
  # take a subset of train_x and train_y
  classifier = KNeighborsClassifier()
  classifier.fit(train_x, train_y)
  return classifier

# decision tree
from sklearn import tree
def decision_tree(train_x, train_y):
  classifier = tree.DecisionTreeClassifier()
  classifier.fit(train_x, train_y)
  return classifier

In [30]:
# run logistic regression
import time
start_time = time.time()
classifier = logistic_regression(X_train, y_train)
log_predictions = classifier.predict(X_test)
print(accuracy_score(y_test, log_predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.7192
--- 2.4119558334350586 seconds ---


In [31]:
# run svm
start_time = time.time()
classifier = svm(X_train, y_train)
predictions = classifier.predict(X_test)
print(accuracy_score(y_test, predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.7352
--- 1293.4701447486877 seconds ---


In [32]:
# run knn
start_time = time.time()
classifier = knn(X_train, y_train)
predictions = classifier.predict(X_test)
print(accuracy_score(y_test, predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.6582
--- 2.195223093032837 seconds ---


In [34]:
# run decision tree
start_time = time.time()
classifier = decision_tree(X_train, y_train)
predictions = classifier.predict(X_test)
print(accuracy_score(y_test, predictions))
print("--- %s seconds ---" % (time.time() - start_time))

0.5977
--- 70.76721978187561 seconds ---
